### UML Species Delimitation Notebook
Author = Alex Pyron; rpyron@colubroid.org

Here we introduce a novel approach to unguided species delimitation using self organizing maps (SOMs), a machine learning (ML) method also known as Kohonon maps. These SOMs reduce multi-dimentional data to a two dimentional configuration that maximizes the similarity between a distance matrix of input and output features. The result is to identify discrete genotype clusters composed of similar individuals. For a full expanation of the theory behind SOMs and an implementation of the method, see Pyron et al. (2022). 

The goal of this notebook is for you to explore this ML method and see if it would be a good fit for your own dataset. Try it out first with the example data from Pyron et al. (2022), then try it with your own structure file and see how it works with your own data. The default parameters are tuned to the example data, so when you use your own data make sure you modify the parameters and check model fit. 

### Install Dependencies
If running on Binder, these are all installed for you within the environment. However, if running locally, you will need to make sure to install.

In [ ]:
#install.packages(c('adegenet','maps','viridis','kohonen'))

In [ ]:
#load dependencies
library(adegenet); library(maps); library(viridis); library(conStruct);library(kohonen)
set.seed(1)

### Read in the data
Here we will read in the structure format data using adegenet. The data has 71 individuals, 7809 loci. If alleles are separated by column, then onrowperind = false, if they are split by line, then make that true. For the SOM method, do no include the population column (col 2), and then make sure to set col.pop = 0. Read more on the [adegenet manual](https://rdrr.io/cran/adegenet/man/read.structure.html).

In [ ]:
#Read in STRUCTURE file from ipyrad
a <- read.structure("./seal_in.str",
                    n.ind = 71,
                    n.loc = 7809,
                    onerowperind = FALSE,
                    col.lab = 1,
                    col.pop = 0,
                    col.others = 0,
                    row.marknames = 0,
                    NA.char = -9)


In [ ]:
# Let's take a quick look
a

Read in the locality information for naming populations and mapping. The input file needed should be a csv that has at minimum the following columns: 
Specimen, mtClade (a priori sp assignment from mtDNA or clustering like Admixture), Lat, Long, State, and County

In [ ]:
# Read in the locality data
dat <- read.csv("./gbs_73_localities.csv")
pops <- factor(dat$mtClade[match(row.names(a$tab),dat$Specimen)])
a$pop <- pops
xy <- dat[,9:8][match(row.names(a$tab),dat$Specimen),]; rownames(xy) <- dat$Specimen[match(row.names(a$tab),dat$Specimen)]
a@other$xy <- xy
a.orig <- a

In [ ]:
# Take a look at the locality data to replicate with your own csv file
head(dat)

In [ ]:
# Convert to a genind object of allele frequencies
struc <- makefreq(a)

In [ ]:
# Take a look at the allele frequencies
head(struc)

In [ ]:
# Convert the allele frequencies to a matrix
struc_matrix <- matrix(unlist(as.numeric(struc)), nrow=nrow(struc))
head(struc_matrix)

In [ ]:
# Convert the data to a grid
size <- round(sqrt(dim(struc)[1]))#round(5 * sqrt(dim(struc_matrix)[1]))
som_grid <- somgrid(xdim = size, ydim=size, topo="hexagonal")
head(som_grid)

### Train the model
Now we have all the input data ready and we can train the model. The input is the data matrix, the grid, and a few parameters. The model doesn't take that long to train, so it is a good idea to experiment with the parameter values for your dataset. 'rlen' = number of generations, 'alpha' = learning rate. As we said above, these parameters are set for the example data, make sure you experiment with your data. For more info on the som model, read [here](https://www.rdocumentation.org/packages/kohonen/versions/2.0.19/topics/som).

In [ ]:
# Train the model
som_model <- som(struc_matrix,
                 grid=som_grid,
                 rlen=350,
                 alpha=c(0.05,0.01),
                 keep.data = TRUE,
                 maxNA.fraction=0.5)

In [ ]:
# Here we plot the model, and you want to find the value where the distance plateaus.
par(mgp=c(3,0.67,0))
plot(som_model, type="changes", main = "a) Training progress", axes=F, ylim=c(0.009,0.015))
axis(1);axis(2,at=c(0.01,0.012,0.014),las=2)

### Identify the right number of clusters

Now we want to use k means clustering to choose the 'best' number of clusters. 

In [ ]:
mydata <- getCodes(som_model)
wss <- (nrow(mydata)-1)*sum(apply(mydata,2,var)) 
for (i in 1:15) {
  wss[i] <- sum(kmeans(mydata, centers=i)$withinss)
}
d_wss <- abs(diff(diff(wss)))
plot(2:14,d_wss, xlab="Clusters (k)",pch=19,ylab="dWSS")
num_clusters <- (2:14)[which(d_wss==max(d_wss))]

### Cluster the inividuals

In [ ]:
## use hierarchical clustering to cluster the codebook vectors
som_cluster <- cutree(hclust(dist(mydata)), num_clusters)
# plot these results:
plot(som_model, shape="straight", type="mapping", bgcol = viridis(num_clusters)[som_cluster], main = "Clusters", 
     pch=19, col="red")#, labels=paste(clade.data$DAB,clade.data$nClade,sep=" "), cex=0.5) 
add.cluster.boundaries(som_model, som_cluster)

In [ ]:
# get vector with cluster value for each original data sample
cluster_assignment <- som_cluster[som_model$unit.classif]
# for each of analysis, add the assignment as a column in the original data:
# data$cluster <- cluster_assignment
cbind(rownames(a$tab),cluster_assignment)
dev.off()

### Map the clusters

In [ ]:
maps::map(database = 'county', xlim = range(xy[,1]) + c(-1,1), ylim = range(xy[,2]) + c(-1,1), col="white")
text(-80,31.5,"c)",cex=1.33,font=2);map.axes()
maps::map(database = 'county', xlim = range(xy[,1]) + c(-1,1), ylim = range(xy[,2]) + c(-1,1), col="gray",add=T)
maps::map(database = 'state', xlim = range(xy[,1]) + c(-1,1), ylim = range(xy[,2]) + c(-1,1), add = T)
#map.scale(x=-81,y=31,relwidth=0.1,ratio=FALSE,cex=0.67)
#legend(x=-82.5,y=33.5,legend=c("monticola B","monticola A/C"),viridis(2))
points(xy[rownames(a$tab),], bg=viridis(2)[cluster_assignment], pch=21, cex=1.25)

### Plot the SOM nodes
In the next figure, the nodes with individuals are colored, whereas the nodes without are gray. Thus, nodes with multiple individuals represent clusters of very similar inidividuals. The color of the node tells you how many individuals are clustered within.

In [ ]:
#Plot the note counts
plot(som_model, type="count", main="Node Counts", palette.name = magma)

In [ ]:
Here, we are plotting the distance between cells. We expect the distances to cluster similar to the individual clusters

In [ ]:
# Plot the Cell distances
plot(som_model, type="dist.neighbours", "SOM neighbour distances", palette.name = magma)

And that is it! Once you run through the example data, play with your own data and see how the learning rate and number of generations affect your results.